In [2]:
'''
Reference:
https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/
'''

'\nReference:\nhttps://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/\n'

In [2]:
#for computer room 

#import nltk
#nltk.download('punkt')
#nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hsnu130427/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hsnu130427/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
#Custom function to process of store:
import json #in 2, this step is indispensable
def process_or_store(tweet):
    print(json.dumps(tweet)) #turn the tweet into a JSON type

# The Anatomy of a Tweet

In [1]:
#單個Tweet裡所包含的所有資訊
import json
with open('Cry.json', 'r') as f:
    line = f.readline()
    tweet = json.loads(line)
    print(json.dumps(tweet, indent=4))
    
#tweet usually in unicode -> transform
enc = tweet['text'].encode('unicode-escape')

import nltk
from nltk import word_tokenize
print(word_tokenize(enc))

#\n 怎麼辦?

{
    "quote_count": 0, 
    "contributors": null, 
    "truncated": false, 
    "text": "RT @choi_bts2: I feel like there will be one more Concept photo tonight.. \ud83e\udd26\ud83c\udffb\u200d\u2640\ufe0f\n\nI think I\u2019ll be late from work but if the Concept photo wi\u2026", 
    "is_quote_status": false, 
    "in_reply_to_status_id": null, 
    "reply_count": 0, 
    "id": 994558244876992512, 
    "favorite_count": 0, 
    "entities": {
        "user_mentions": [
            {
                "id": 953663868659798017, 
                "indices": [
                    3, 
                    13
                ], 
                "id_str": "953663868659798017", 
                "screen_name": "choi_bts2", 
                "name": "Soo Choi"
            }
        ], 
        "symbols": [], 
        "hashtags": [], 
        "urls": []
    }, 
    "retweeted": false, 
    "coordinates": null, 
    "timestamp_ms": "1525956138234", 
    "source": "<a href=\"http://twitter.com/download

['RT', '@', 'choi_bts2', ':', 'I', 'feel', 'like', 'there', 'will', 'be', 'one', 'more', 'Concept', 'photo', 'tonight..', '\\U0001f926\\U0001f3fb\\u200d\\u2640\\ufe0f\\n\\nI', 'think', 'I\\u2019ll', 'be', 'late', 'from', 'work', 'but', 'if', 'the', 'Concept', 'photo', 'wi\\u2026']


In [2]:
#the tweet be divided into too seperate
import re

emoticons_str = r'''
    (?:
        [:=;] #eye
        [oO\-] #nose
        [D\)\]\)\]/\\OpP] #mouth
    )'''

regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
    
    #r'' #\n
]

#re.compile(pattern) comile the pattern of re.
#ref: http://zwindr.blogspot.jp/2016/01/python-regular-expression.html
tokens_re = re.compile(r'(' + '|'.join(regex_str) + ')', re.X |re.I)
emoticon_re = re.compile(r'^' + emoticons_str + '$', re.X | re.I)
#I -> ignorance: take the lowercase and uppercase as same
#X -> Verbose: make a re. more readable, ignore the '#' explantion


def tokenize(s):
    return tokens_re.findall(s)

def preprocess(s, lowercase=False):
    tokens = tokenize(s.encode('unicode-escape'))
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower for token in tokens]
    return tokens

    
#re.search:Scan through string looking for the first 
#      location where the regular expression pattern 
#      produces a match

#pre_enc = preprocess(enc)
#print(pre_enc) 

#### 目前已完整分解字詞，並可獨立出emoji，接下來做成dataframe

In [3]:
word_list = []
with open('Cry.json') as f:
    for line in f:
        tweet = json.loads(line)
        text = tweet['text'].encode('unicode-escape')
        words = preprocess(text) #只留下重要字詞，需要重寫函數
        word_list.append(words) #一個超長list

from itertools import chain
word_list = list(chain.from_iterable(word_list))
word_list = [word.lower() for word in word_list] #斷完詞後的"所有"詞跟符號

print(word_list)

['rt', '@choi_bts2', ':', 'i', 'feel', 'like', 'there', 'will', 'be', 'one', 'more', 'concept', 'photo', 'tonight', '.', '.', '\\', '\\', 'u0001f926', '\\', '\\', 'u0001f3fb', '\\', '\\', 'u200d', '\\', '\\', 'u2640', '\\', '\\', 'ufe', '0', 'f', '\\', '\\', 'n', '\\', '\\', 'ni', 'think', 'i', '\\', '\\', 'u2019ll', 'be', 'late', 'from', 'work', 'but', 'if', 'the', 'concept', 'photo', 'wi', '\\', '\\', 'u2026', 'rt', '@faiqyuhaizad', ':', 'for', 'a', 'better', 'malaysia', ',', 'jangan', 'mengharapkan', 'perubahan', 'daripada', 'kerajaan', 'semata', '2,', 'rakyat', 'kena', 'berubah', 'gak', '.', 'sampah', 'jangan', 'bu', '\\', '\\', 'u2026', 'rt', '@hafizkrs', ':', 'im', 'gonna', 'miss', 'this', 'guy', '\\', '\\', 'u0001f622', 'https://t.co/qrkxzoxhwq', 'rt', '@iam_khunguy', ':', '*', '\\', '\\', 'u0e1d', '\\', '\\', 'u0e32', '\\', '\\', 'u0e01', '\\', '\\', 'u0e41', '\\', '\\', 'u0e17', '\\', '\\', 'u0e49', '\\', '\\', 'u0e01', '\\', '\\', 'u0e2b', '\\', '\\', 'u0e19', '\\', '\\', 'u0

### Build dataframe of each word and emoji 
### Take the unique word and emoji from tweet 

In [17]:
from nltk.corpus import stopwords
import string

punctuation = [string.punctuation]
stop_uni = stopwords.words('english') + punctuation + ['rt','via']

stop = [word.encode('ascii') for word in stop_uni]    
word_stop = [word for word in word_list 
            if word not in stop and
                    not word.startswith(
    ('u0','http','@','#','\\','/','*','-','{','}','[',']','(',')',
     '+','x',':',"''",'i\\','.','?','!',"'",'"',',',';',':','&',''
     'u2','ua','ub','uc','ud')) and
                    not word.isdigit()]
emoji = [emoji[:9] for emoji in word_list #there's a case is u000f162somebody
             if emoji.startswith(('u000'))]

In [5]:
#build the dataframe
word_single = list(set(word_stop))
emoji_single = list(set(emoji))

word_single.append('somebody')
word_single = sorted(word_single)

import pandas as pd
import numpy as np

#pd.DataFrame(0,
#             columns = emoji_single,
#             index = word_single)

emoji_single

['u0001f602',
 'u0001f622',
 'u0001f614',
 'u0001f3fb',
 'u0001f926',
 'u0001f937',
 'u0001f3fe',
 'u0001f494',
 'u0001f319',
 'u0001f62d',
 'u0001f60d',
 'u0001f942',
 'u0001f631',
 'u0001f499']

In [6]:
import pandas as pd
first_df = pd.DataFrame([np.zeros(len(emoji_single), dtype=np.int8)],
            index = word_single,columns = emoji_single)

## Method1: Freq to build dataframe

In [7]:
import nltk
from nltk import word_tokenize
from nltk.probability import FreqDist
import pandas as pd

def text_process(text):
    
    enc = text.encode('unicode-escape')
    pro_text = preprocess(enc)

    #[emoji for emoji in pro_text if emoji.startswith('U000')]
    stop_tuple = ('u00','http','@','#','\\','/','*','-','{','}','[',']','+','x',':',"''",'i\\','.','?','!',"'",'"',
     ',',';',':','&',''
     'u2','ua','ub','uc','ud')
    emoji_list = [emoji.lower() for emoji in pro_text if emoji.startswith('U00')]
    pro_text = [word.lower() for word in pro_text if word not in stop #'in', so 'word not in'
                            and not word.startswith(stop_tuple)
                            and not word.isdigit()
                            and not len(word) < 2]
    return emoji_list, FreqDist(pro_text)

In [8]:
import json
import numpy as np
cry_list = []

with open('Cry.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        emoji_list, pro_text = text_process(tweet['text'])
        cry_list.append([emoji_list, pro_text])
        #df = merge_dataframe(emoji_list, pro_text, df_emoji)

cry_list

[[['u0001f926', 'u0001f3fb'],
  FreqDist({'concept': 2,
            'feel': 1,
            'late': 1,
            'like': 1,
            'ni': 1,
            'one': 1,
            'photo': 2,
            'rt': 1,
            'think': 1,
            'tonight': 1,
            'u0001f3fb': 1,
            'u0001f926': 1,
            'ufe': 1,
            'wi': 1,
            'work': 1})],
 [[],
  FreqDist({'2,': 1,
            'berubah': 1,
            'better': 1,
            'bu': 1,
            'daripada': 1,
            'for': 1,
            'gak': 1,
            'jangan': 2,
            'kena': 1,
            'kerajaan': 1,
            'malaysia': 1,
            'mengharapkan': 1,
            'perubahan': 1,
            'rakyat': 1,
            'rt': 1,
            'sampah': 1,
            'semata': 1})],
 [['u0001f622'],
  FreqDist({'gonna': 1,
            'guy': 1,
            'im': 1,
            'miss': 1,
            'rt': 1,
            'u0001f622': 1})],
 [['u0001f622'],
  Freq

# Method2: Use For Loop to loop each sentence 

In [30]:
import nltk
from nltk import word_tokenize

def text_process_simple(text):
    
    enc = text.encode('unicode-escape')
    pro_text = preprocess(enc)

    #[emoji for emoji in pro_text if emoji.startswith('U000')]
    stop_tuple = ('U000','RT','@',':','/','\\''','.','u2','http','u0','#','xf','xe','('
                 'ua','ub','uc','ud')
    emoji_list = [emoji.lower()[:9] for emoji in pro_text if emoji.startswith('U00')
                 ]
    pro_text = [word.lower() for word in pro_text]
    pro_text = [word.lower() for word in pro_text if not word.startswith(stop_tuple)
                            and word not in stop
                            #and word in word_single 之前是根據df來篩，整體重用就不需要
                            #and word.lower() not in stop #some He, Her, Is aren't captured by stoplist
                            and not word.isdigit()
                            and not len(word) < 2]
    return emoji_list, pro_text

In [10]:
import json
import numpy as np
cry_list = []

with open('Cry.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        emoji_list, pro_text = text_process_simple(tweet['text'])
        cry_list.append([emoji_list, pro_text])
        #df = merge_dataframe(emoji_list, pro_text, df_emoji)

for i in cry_list:
    for j in i[0]:
        first_df.loc[i[1],j] += 1

In [14]:
#first_df
#np.argsort(first_df.u0001f622) order from a~z with the num.

In [33]:
import nltk
from nltk import word_tokenize

def temp_process(text):
    
    enc = text.encode('unicode-escape')
    tokens = word_tokenize(enc)
    
    return tokens

angry_list = []

with open('Angry.json', 'r') as f:
    for line in f:
        tweet = json.loads(line)
        print(temp_process(tweet['text']))
        #emoji_list, pro_text = text_process_simple(tweet['text'])
        #angry_list.append([emoji_list, pro_text])

angry_list
#angry_list #losing emoji in the first index?!!!

['Instagrami', 'da', '\\xe7\\xf6kerttiniz', 'be', '\\U0001f621']
['Avevo', 'fatto', 'un', 'post', 'molto', 'bello', 'e', '...', '#', 'instagramdown', '\\U0001f621']
['RT', '@', 'rajeev_mp', ':', 'Trying', 'to', 'win', '#', 'Elections', '@', 'INCIndia', 'style', '-', 'when', 'u', 'cant', 'get', 'real', 'votes', '-', 'create', 'fake', 'voters/votes', '\\U0001f621\\n\\n', '#', 'SarkaraBadalisiBJPGelli\\u2026']
['Yeah', 'there', "'s", 'video', 'of', 'kaines', 'kid', 'resisting', 'arrest', 'and', 'smoke', 'bombing', 'the', 'capitol', '.', 'Slap', 'on', 'the', 'wrist', '.', 'Privileged', 'clas\\u2026', 'https', ':', '//t.co/3E4SuPI7k3']
['@', 'elyszian', '@', 'keyzazvn', '@', 'alsheirazakhtar', '@', 'brxkendxlls', '@', 'MTUANDSM', '@', 'DOYEON30C', '@', 'albarysaptr_', '@', 'ladybjoohyun', '@', '51sYOONA\\u2026', 'https', ':', '//t.co/Wy1v9cpBjr']
['RT', '@', 'Sporf', ':', '\\U0001f4de', 'Caller', ':', '\\u201cMorata', 'should', 'be', 'giving', 'it', 'his', 'all', ',', 'as', 'he', 'hasn\\u20

[]

In [49]:
import json
emo_list = []

with open('Emoji.json') as f:
    for line in f:
        tweet = json.loads(line)
        if len(tweet) > 20:
            emoji_list, pro_text = text_process_simple(tweet['text'])
            emo_list.append([emoji_list, pro_text])
            #print(preprocess(tweet['text']))
            #print(tweet['text'])

In [50]:
emo_list

[[['u0001f917'], ['love', 'everyone', 'wants', 'meet']],
 [['u0001f602'], ['hope', "there's", 'trash', 'tweet']],
 [['u0001f618'], ['punish', 'pussy', 'give', 'kiss']],
 [['u0001f629'], ['hands', 'yoongi', 'hobi', 'gotta', 'burn', 'shirts']],
 [['u0001f602'],
  ['dak',
   'prescott',
   'th',
   'best',
   'qb',
   'nfc',
   'east',
   'nhttps',
   'co',
   'flbx',
   'fnrfe']],
 [['u0001f609'],
  ['highlights',
   'harshad',
   'chopda',
   'birthday',
   'party',
   'fans',
   'watch',
   'end',
   'special',
   'treat']],
 [['u0001f171', 'u0001f62d'],
  ['someone', 'said', 'ufe', 'fareem', 'hq', 'comments']],
 [[],
  ['hell',
   'nah',
   'friends',
   'gonna',
   'try',
   'funny',
   'rock',
   'boat',
   'everyone',
   'gonna',
   'flip',
   'next',
   'thing',
   'know',
   'jaws']],
 [[],
  ['fucking', 'hell', 'looks', 'like', 'dug', 'could', 'attend', 'wedding']],
 [['u0001f602'], ['hope', "there's", 'trash', 'tweet']],
 [['u0001f602'],
  ['still',
   'single',
   'nme',
   'n

In [71]:
word_single = []
emoji_single = []

for i in range(len(emo_list)):
    word_single.append(emo_list[i][1])
    emoji_single.append(emo_list[i][0])
    
word_single = list(sorted(set([word for word_list in word_single 
                        for word in word_list])))
emoji_single = list(set([emoji for emoji_list in emoji_single 
                              for emoji in emoji_list]))

word_single

['0.040548',
 '0.117830',
 '0.123837',
 '1.00',
 '1.54',
 '1.69',
 '10,000',
 '100,000',
 '15,',
 '15,000',
 '15,175,000',
 '16,',
 '169,000',
 '18,',
 '19,',
 '19.2',
 '1995,',
 '2,000',
 '2.1',
 '2.13',
 '2.2',
 '2.69',
 '2.9',
 '22,000',
 '3,000',
 '31,',
 '385.3',
 '39,500',
 '4,',
 '5,000',
 '5.88',
 '52,100',
 '7,',
 '8,',
 '8.50',
 '800,000',
 '819,000',
 '9.00',
 ';-)',
 '_9',
 '___________________________________________',
 'aa',
 'aaaaaaa',
 'aaaaaaaaa',
 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaah',
 'aaaaaahhh',
 'aaaaah',
 'aaaagh',
 'aahhhh',
 'aap',
 'ab',
 'abarklk',
 'abby',
 'abc',
 'abetting',
 'abhay',
 'able',
 'absolute',
 'absolutely',
 'absurd',
 'abt',
 'abu',
 'abuse',
 'aby',
 'abyad',
 'aca',
 'acc',
 'accept',
 'acceptance',
 'accepted',
 'accepting',
 'accepts',
 'accessorie',
 'accessory',
 'accident',
 'according',
 'account',
 'accountable',
 'accounts',
 'acct',
 'accuracy',
 'accurate',
 'accus',
 'accuse',
 'ace',
 'achieve',
 'acknowledge',
 'acoustics

In [74]:
#build the dataframe
emoji_df = pd.DataFrame([np.zeros(len(emoji_single), dtype=np.int64)], 
            columns = emoji_single, index = word_single)

for i in emo_list:
    for j in i[0]:
        print(emoji_df.loc[i[1], j])
        #emoji_df.loc[i[1],j] += 1

love        0
everyone    0
wants       0
meet        0
Name: u0001f917, dtype: int64
hope       0
there's    0
trash      0
tweet      0
Name: u0001f602, dtype: int64
punish    0
pussy     0
give      0
kiss      0
Name: u0001f618, dtype: int64
hands     0
yoongi    0
hobi      0
gotta     0
burn      0
shirts    0
Name: u0001f629, dtype: int64
dak         0
prescott    0
th          0
best        0
qb          0
nfc         0
east        0
nhttps      0
co          0
flbx        0
fnrfe       0
Name: u0001f602, dtype: int64
highlights    0
harshad       0
chopda        0
birthday      0
party         0
fans          0
watch         0
end           0
special       0
treat         0
Name: u0001f609, dtype: int64
someone     0
said        0
ufe         0
fareem      0
hq          0
comments    0
Name: u0001f171, dtype: int64
someone     0
said        0
ufe         0
fareem      0
hq          0
comments    0
Name: u0001f62d, dtype: int64
hope       0
there's    0
trash      0
tweet      

Name: u0001f602, dtype: int64
ambisi         0
ng             0
fandom         0
mainelandia    0
july           0
adn            0
food           0
fest           0
july           0
adn            0
soir           0
cebu           0
july           0
adn            0
homecoming     0
Name: u0001f618, dtype: int64
showed       0
sec          0
cheering     0
sorry        0
muted        0
sound        0
cause        0
screaming    0
like         0
crazy        0
Name: u0001f602, dtype: int64
omg        0
eargasm    0
max        0
Name: u0001f61a, dtype: int64
turning    0
makes      0
feel       0
old        0
got        0
solid      0
years      0
Name: u0001f602, dtype: int64
aunty    0
right    0
gone     0
lmao     0
Name: u0001f602, dtype: int64
car        0
talking    0
shit       0
ncar       0
little     0
bitch      0
nme        0
quit       0
cry        0
baby       0
pussy      0
ncar       0
fuck       0
skin       0
tough      0
Name: u0001f612, dtype: int64
car        0
tal

Name: u0001f62d, dtype: int64
sorry        0
dnwmdnenw    0
forgot       0
Name: u0001f614, dtype: int64
got     0
love    0
boys    0
Name: u0001f609, dtype: int64
bts           0
announced     0
performers    0
look          0
proud         0
Name: u0001f60d, dtype: int64
bts         0
taylor      0
swift       0
universe    0
Name: u0001f633, dtype: int64
congrats    0
kyle        0
surprise    0
Name: u0001f1fa, dtype: int64
congrats    0
kyle        0
surprise    0
Name: u0001f1f8, dtype: int64
congrats    0
kyle        0
surprise    0
Name: u0001f388, dtype: int64
congrats    0
kyle        0
surprise    0
Name: u0001f642, dtype: int64
nicholas    0
arms        0
jesus       0
Name: u0001f60d, dtype: int64
disrespect    0
wob           0
like          0
Name: u0001f621, dtype: int64
lmao      0
thanks    0
okay      0
Name: u0001f62d, dtype: int64
nick       0
fucking    0
fine       0
Name: u0001f61b, dtype: int64
meet         0
another      0
camilizer    0
Name: u0001f609, dtyp

cheesing    0
watching    0
videos      0
moe         0
weirdo      0
Name: u0001f602, dtype: int64
would        0
love         0
adventure    0
depths       0
marvel       0
Name: u0001f60d, dtype: int64
showed       0
sec          0
cheering     0
sorry        0
muted        0
sound        0
cause        0
screaming    0
like         0
crazy        0
Name: u0001f602, dtype: int64
talking    0
red        0
carpet     0
xa9        0
ufe        0
Name: u0001f60d, dtype: int64
talking    0
red        0
carpet     0
xa9        0
ufe        0
Name: u0001f60d, dtype: int64
love        0
everyone    0
wants       0
meet        0
Name: u0001f917, dtype: int64
dude    0
mind    0
Name: u0001f633, dtype: int64
good        0
workout     0
erection    0
always      0
good        0
Name: u0001f608, dtype: int64
meet         0
another      0
camilizer    0
Name: u0001f609, dtype: int64
saying       0
chris        0
said         0
following    0
around       0
like         0
damn         0
lying    

soooooooooo    0
got            0
better         0
seats          0
ask            0
rm             0
got            0
Name: u0001f602, dtype: int64
soooooooooo    0
got            0
better         0
seats          0
ask            0
rm             0
got            0
Name: u0001f602, dtype: int64
soooooooooo    0
got            0
better         0
seats          0
ask            0
rm             0
got            0
Name: u0001f602, dtype: int64
fuckery    0
ufe        0
seen       0
dawg       0
ruining    0
ed         0
edd        0
eddy       0
Name: u0001f621, dtype: int64
travel     0
world      0
exo's      0
laddar     0
airing     0
oksusu     0
kst        0
use        0
show       0
support    0
Name: u0001f606, dtype: int64
misses       0
miss         0
continues    0
deliver      0
paper        0
Name: u0001f623, dtype: int64
misses       0
miss         0
continues    0
deliver      0
paper        0
Name: u0001f643, dtype: int64
new         0
ninja       0
case        0
new    

...

Name: u0001f621, dtype: int64
already         0
blaming         0
progressives    0
democrats       0
losing          0
good            0
dem             0
Name: u0001f621, dtype: int64
already         0
blaming         0
progressives    0
democrats       0
losing          0
good            0
dem             0
Name: u0001f621, dtype: int64
already         0
blaming         0
progressives    0
democrats       0
losing          0
good            0
dem             0
Name: u0001f621, dtype: int64
already         0
blaming         0
progressives    0
democrats       0
losing          0
good            0
dem             0
Name: u0001f621, dtype: int64
already         0
blaming         0
progressives    0
democrats       0
losing          0
good            0
dem             0
Name: u0001f621, dtype: int64
already         0
blaming         0
progressives    0
democrats       0
losing          0
good            0
dem             0
Name: u0001f621, dtype: int64
already         0
blaming         

Name: u0001f641, dtype: int64
friendship    0
lovely        0
Name: u0001f60a, dtype: int64
look      0
outfit    0
Name: u0001f60d, dtype: int64
never       0
get         0
tired       0
watching    0
videos      0
maine       0
nour        0
nominee     0
nth         0
Name: u0001f602, dtype: int64
friendship    0
lovely        0
Name: u0001f60a, dtype: int64
hope       0
there's    0
trash      0
tweet      0
Name: u0001f602, dtype: int64
acrobatics    0
Name: u0001f631, dtype: int64
watching    0
carpool     0
karaoke     0
paused      0
moment      0
nharry's    0
faces       0
Name: u0001f602, dtype: int64
yanny        0
laurel       0
stretcher    0
Name: u0001f629, dtype: int64
i'm         0
never       0
going       0
drinking    0
gotta       0
work        0
next        0
day         0
takes       0
old         0
ass         0
long        0
recover     0
Name: u0001f602, dtype: int64
i'm         0
never       0
going       0
drinking    0
gotta       0
work        0
next     

Name: u0001f927, dtype: int64
probably    0
proudest    0
moment      0
twitter     0
getting     0
like        0
Name: u0001f62d, dtype: int64
got     0
love    0
boys    0
Name: u0001f609, dtype: int64
joon        0
really      0
audience    0
shades      0
Name: u0001f602, dtype: int64
joon        0
really      0
audience    0
shades      0
Name: u0001f60e, dtype: int64
showed       0
sec          0
cheering     0
sorry        0
muted        0
sound        0
cause        0
screaming    0
like         0
crazy        0
Name: u0001f602, dtype: int64
dates     0
bbmas     0
couple    0
goals     0
Name: u0001f60d, dtype: int64
always         0
laying         0
edge           0
bed            0
left           0
leg            0
hanging        0
phone          0
right          0
hand           0
left           0
arm            0
chin           0
comfortable    0
Name: u0001f60c, dtype: int64
phone      0
charger    0
phone      0
says       0
battery    0
Name: u0001f62b, dtype: int64
min

Name: u0001f622, dtype: int64
noo        0
fucking    0
kapp       0
real       0
yellow     0
beak       0
orange     0
feet       0
Name: u0001f986, dtype: int64
noo        0
fucking    0
kapp       0
real       0
yellow     0
beak       0
orange     0
feet       0
Name: u0001f602, dtype: int64
got       0
baby      0
turtle    0
idk       0
name      0
Name: u0001f61e, dtype: int64
wow    0
Name: u0001f60d, dtype: int64
gf    0
Name: u0001f608, dtype: int64
heart           0
melted          0
bts             0
hugged          0
chainsmokers    0
ufe             0
ufe             0
ufe             0
Name: u0001f62d, dtype: int64
got     0
love    0
boys    0
Name: u0001f609, dtype: int64
meet         0
another      0
camilizer    0
Name: u0001f609, dtype: int64
friendship    0
lovely        0
Name: u0001f60a, dtype: int64
friendship    0
lovely        0
Name: u0001f60a, dtype: int64
friendship    0
lovely        0
Name: u0001f60a, dtype: int64
bts          0
ones         0
stood     

Name: u0001f602, dtype: int64
talking    0
red        0
carpet     0
xa9        0
ufe        0
Name: u0001f60d, dtype: int64
baby      0
boy       0
snacks    0
Name: u0001f60d, dtype: int64
trash      0
call       0
bell       0
harden     0
end        0
half       0
explain    0
foul       0
even       0
contend    0
shot       0
Name: u0001f914, dtype: int64
icymi     0
red       0
carpet    0
Name: u0001f631, dtype: int64
lmao        0
called      0
drank       0
somebody    0
warned      0
amp         0
amp         0
listen      0
Name: u0001f602, dtype: int64
lmao        0
called      0
drank       0
somebody    0
warned      0
amp         0
amp         0
listen      0
Name: u0001f480, dtype: int64
time         0
another      0
hour         0
confusion    0
Name: u0001f923, dtype: int64
like    0
Name: u0001f60d, dtype: int64
y'all       0
playing     0
much        0
fortnite    0
Name: u0001f602, dtype: int64
niiiiicccceee    0
Name: u0001f601, dtype: int64
happen     0
feel    

In [73]:
emo_list[0][0]

['u0001f917']

In [67]:
emoji_df

,u0001f445,u0001f98b,u0001f3df,u0001f48a,u0001f3dd,u0001f5a5,u0001f5a4,u0001f918,u0001f601,u0001f600,...,u0001f346,u0001f502,u0001f503,u0001f478,u0001f4c5,u0001f6ab,u0001f4b5,u0001f46f,u0001f35f,u0001f51e
raining,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
foul,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
four,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
hanging,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
un-guardable,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
conjure,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
looking,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
canes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
fanmeet,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bitchh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
